In [2]:
import numpy as np
import random

### Variables globales del problema

In [ ]:
# VERTEX COVER - COBERTURA DE VÉRTICES

import networkx as nx
#G = nx.read_edgelist(r"..\vertex_cover_benchmarks\ENZYMES_g102.edges", nodetype=int)
#G = nx.read_edgelist(r"..\vertex_cover_benchmarks\aves-sparrowlyon-flock-season2-unw.edges", nodetype=int)
#G = nx.read_edgelist("brock200_1_complement.edges", nodetype=int)
#G = nx.read_edgelist(r"..\vertex_cover_benchmarks\sparrow_unweighted.edges", nodetype=int)
G = nx.read_edgelist(r"..\vertex_cover_benchmarks\cats.edges", nodetype=int)
n = G.order()

In [3]:
# KNAPSACK - MOCHILA

values = []
weights = []

with open(r"..\knapsack_benchmarks\s000.kp") as f:
    n, W = [int(x) for x in next(f).split()]
    for line in f:
        nums = line.split()
        values.append(int(nums[0]))
        weights.append(int(nums[1]))

sum_values = sum(values)

### Parámetros

In [4]:
MAX_FITNESS = 1
MAX_ITER = 5000
SAMPLE_SIZE = 100
INITIAL_SAMPLE_SIZE = 300

### Clase Solution

In [5]:
class Solution:
    def __init__(self, sol, f = 0):
        self.sol = sol
        self.f = f

    def set_f(self, f):
        self.f = f

### Funciones auxiliares

In [ ]:
def is_cover(sol, G):
    for u, v in G.edges():
        if sol[u - 1] == 0 and sol[v - 1] == 0:
            return False
    return True

def naive_cover(sol, G):
    for u, v in G.edges():
        if sol[u - 1] == 0 and sol[v - 1] == 0:
            node = random.choice([u, v])
            sol[node - 1] = 1
    return

def fitness_vertex_cover(sol):
    n = len(sol)
    if not is_cover(sol, G):
        naive_cover(sol, G)
    return (n - sum(sol)) / sum(sol)

In [6]:
def fitness_knapsack(sol):
    n = len(sol)
    fit = 0
    wei = 0
    for i in range(n):
        fit += values[i] * sol[i]
        wei += weights[i] * sol[i]
    while wei > W:
        sacar = random.randrange(0, n)
        wei -= weights[sacar] * sol[sacar]
        fit -= values[sacar] * sol[sacar]
        sol[sacar] = 0
    return fit / sum_values

In [7]:
def f(sol, fitness_func):
    fit = fitness_func(sol.sol)
    sol.set_f(fit)
    return fit

In [8]:
def hamming_distance(sol1, sol2):
    n = len(sol1)
    assert(n == len(sol2))
    return [sol1[i] == sol2[i] for i in range(n)].count(False) ** 3

In [9]:
def inverse_distance(sol1, sol2, dist):
    d = dist(sol1, sol2)
    return 1 / d if d != 0 else 2 # Realmente la forma de manejar el infinito depende de la distancia
# El 2 funciona para Hammings porque no puede haber distancias d 0 < d < 1 (es decir, tales que 1 / d > 2).

In [10]:
def w(sol1, sol2, A, dist):
    assert(len(sol1) == len(sol2))
    sum = 0
    for point in A:
        sol = point.sol
        assert(len(sol) == len(sol1))
        sum += inverse_distance(sol1, sol, dist)
    assert(sum > 0) # Creo que se puede demostrar que esto no va a ser 0.
    inv = inverse_distance(sol1, sol2, dist)
    res = inv / sum
    return res

def g(sol, A, dist):
    sum = 0
    for point in A:
        wei = w(sol.sol, point.sol, A, dist)
        fit = point.f
        sum += wei * fit
    return sum

In [11]:
def search_optimal(A):
    card = len(A)
    assert(card > 0)
    opt = A[0]
    max_fit = opt.f
    for elem in A[1:]:
        fit = elem.f
        if fit > max_fit:
            max_fit = fit
            opt = elem
    return opt

def search_nearest(sol, A, dist):
    card = len(A)
    assert(card > 0)
    nearest = A[0]
    min_distance = dist(sol.sol, nearest.sol)
    for elem in A[1:]:
        distance = dist(sol.sol, elem.sol)
        if distance < min_distance:
            min_distance = distance
            nearest = elem
    return nearest

def search_2_nearest(sol, A, dist):
    card = len(A)
    assert(card > 1)
    first_distance = dist(sol.sol, A[0].sol)
    second_distance = dist(sol.sol, A[1].sol)
    if first_distance > second_distance:
        nearest = A[1]
        second = A[0]
        min_distance = second_distance
        sec_min_distance = first_distance
    else:
        nearest = A[0]
        second = A[1]
        min_distance = first_distance
        sec_min_distance = second_distance
    for elem in A[2:]:
        distance = dist(sol.sol, elem.sol)
        if distance < sec_min_distance:
            if distance < min_distance:
                sec_min_distance = min_distance
                min_distance = distance
                second = nearest
                nearest = elem
            else:
                sec_min_distance = distance
                second = elem
    return [nearest, second]

### Lógica del algoritmo generalizado

In [12]:
def sample(k, A, dist, dimension, fitness_func, skip = []):
    sample = []
    while len(sample) < k:
        x = [random.choice([0, 1]) for _ in range(dimension)]
        if x in sample or x in skip:
            continue
        y = random.uniform(0, MAX_FITNESS)
        sol = Solution(x)

        # Para GA:
        set_points = search_2_nearest(sol, A, dist)

        # Para PSO:
        #set_points = [search_optimal(A), search_nearest(sol, A, dist)]

        # Para híbrido GA+PSO:
        #set_points = [search_optimal(A)] + search_2_nearest(sol, A, dist)

        g_val = g(sol, set_points, dist)
        if g_val > y:
            f(sol, fitness_func)
            sample.append(sol)
    return sample

def init(k, dimension, fitness_func):
    init = []
    while len(init) < k:
        x = [random.choice([0, 1]) for _ in range(dimension)]
        if not any(point.sol == x for point in init):
            sol = Solution(x)
            f(sol, fitness_func)
            init.append(sol)
    return init

def max_f(A):
    assert(len(A) > 0)
    max_value = A[0].f
    max_elem = A[0]
    for elem in A[1:]:
        fit = elem.f
        if fit > max_value:
            max_value = fit
            max_elem = elem
    return max_elem, max_value

def next_swarm_step(k, A, dist, dimension, fitness_func):
    opt = search_optimal(A)
    next = sample(k - 1, A, dist, dimension, fitness_func, skip = A)
    next.append(opt)
    return next

def algorithm(sample_size, num_iter, dist, dimension, fitness_func):
    A = init(sample_size, dimension, fitness_func)
    for i in range(num_iter):
        A = next_swarm_step(sample_size, A, dist, dimension, fitness_func)
    return max_f(A)

In [13]:
output = open(r"..\results\kn0\medias_ponderadas_GA.csv", "a")
for i in range(20):
    print(f"ejec {i}")
    elem, _ = algorithm(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, dist = hamming_distance, dimension = n, fitness_func = fitness_knapsack)
    output.write(f'{elem.sol},')
    fit = 0
    for i in range(n):
        fit += values[i] * elem.sol[i]
    #num_ceros = G.order() - sum(elem.sol)
    print(fit)
    output.write(f'{fit}\n')
output.close()

ejec 0
18931
ejec 1
19042
ejec 2
19204
ejec 3
19110
ejec 4
19098
ejec 5
19025
ejec 6
19277
ejec 7
18819
ejec 8
19079
ejec 9
18846
ejec 10
19490
ejec 11
19106
ejec 12
19107
ejec 13
18927
ejec 14
19186
ejec 15
19609
ejec 16
18762
ejec 17
18884
ejec 18
19085
ejec 19
19452
